Topic Modeling Using Distributed Word Embeddings
================================================
Notebook version of https://github.com/rsrandhawa/Vec2Topic code, based on the article "Topic Modeling Using Distributed Word Embeddings" by R. S. Randhawa, P. Jain, and G. Madan. 

The basic approach is to first create a language model based on a large (ideally billions of words) text corpus. The technology used, distributed word embeddings, is a shallow neural network that seems to perform best on large datasets (trades simple but fast computation for tons of data).

The user generated content (which is usually a much smaller corpus) is likewise trained with consistent parameters. Vectors corresponding to the same vocabulary word are concatenated together to provide a model of the user generated content.

Word vectors that cluster together are interperted as topics of the user generated content. Some clusters appear better than others because they consist of coherent lists of words -- main goal is to score the importance of each topic.

Performing a hierarchical clustering provides a measure of depth for each word and computing a co-occurance graph (edge between two words if they belong to the same sentenence) provides a degree of co-occurance. Each word is scored by a (normalized) product of depth and degree. KMeans is used to cluster words into topics, and the scoring function is used to order the words and the topics.

Required standard packages
--------------------------

In [ ]:
import logging, re, os, bz2
from collections import Counter
from operator import itemgetter
import itertools

In [ ]:
## Unicode wrapper for reading & writing csv files.
import unicodecsv as csv

## Lighter weight than pandas -- tabular display of tables.
from tabulate import tabulate

## In order to strip out the text from the xml formated data.
from bs4 import BeautifulSoup
import lxml

Required data science packages
------------------------------

In [ ]:
## First the usual suspects: numpy, scipy, and gensim
import numpy as np
import scipy as sp
import gensim

## For scraping text out of a wikipedia dump. Get dumps at https://dumps.wikimedia.org/backup-index.html
from gensim.corpora import WikiCorpus

## Latest greatest word vectors (see https://pypi.python.org/pypi/fasttext).
import fasttext

## Latest greatest hierarchical clustering package. 
## Word vectors are clustered, with deeper trees indicating core topics.
import fastcluster

## Use scikit-learn to generate co-occurancy graph (edge if words in same sentence).
## The degree of each word indicates how strong it co-occurs.
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

## Use scikit-learn for K-Means clustering: identify topics.
from sklearn.cluster import KMeans

Base data directory and logging.
--------------------------------
The approach currently uses a lot of intermediate files (which is annoying, but means that the project can work on machines with smaller physical memory). The initial data (knowledge base as well as user generated content) and the intermediate files are all kept in the data directory.

In [ ]:
data_directory = 'data/'
model_directory = 'models/'

In [ ]:
from imp import reload
reload(logging)

LOG_FILENAME = data_directory + 'vec2topic.log'
#logging.basicConfig(filename=LOG_FILENAME,level=logging.INFO)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s %(message)s',"%b-%d-%Y %H:%M:%S")
logger.handlers[0].setFormatter(formatter)

List of intermediate files.
---------------------------
The (global) knowledge base is built off a (large) dataset.

In [ ]:
if False:
    knowledge_base = 'viwiki-20160920-pages-articles.xml.bz2'
else:
    knowledge_base = 'vie_newscrawl_2011_1M.tkn.wseg'
    
knowledge_base_vector_dimension = 300    # Word vector dimensionality for knowledge base.
knowledge_base_prefix = 'vie_newscrawl_2011_1M'


knowledge_base_text = data_directory + knowledge_base
knowledge_base_phrases = data_directory + knowledge_base_prefix + '_phrases.txt'

knowledge_base_model = model_directory + knowledge_base_prefix + '.bin'
knowledge_base_vectors = model_directory + knowledge_base_prefix + '.vec'
knowledge_base_vectors_tsne = model_directory + knowledge_base_prefix + '_vec_tsne.txt'
knowledge_base_vocab = model_directory + knowledge_base_prefix + '_vocab.txt'

The (local) user generated content.

In [ ]:
local_content_name = 'OpenSubtitles2016_xml_vi_2015_369610_6346303'
local_content_vector_dimension = 25

local_content = data_directory + local_content_name

## Intermediate files associated in proccessing input with external Java package JVnTextPro.
local_content_xml = local_content + '.xml'
local_content_txt = local_content + '.txt'
local_content_txt_sent = local_content + '.txt.sent'
local_content_txt_sent_tkn = local_content + '.txt.sent.tkn'
local_content_txt_sent_tkn_wseg = local_content + '.txt.sent.tkn.wseg'
local_content_txt_sent_tkn_wseg_pos = local_content + '.txt.sent.tkn.wseg.pos'

## Intermediate files resulting from computation of word embeddings using fastText package.
local_content_vectors = model_directory + local_content_name + '.vec'
local_content_model = model_directory + local_content_name + '.bin'

## Projected 2D vectors useful for visualization.
local_content_vectors_tsne = model_directory + local_content_name + '_vec_tsne.txt'

In [ ]:
combined_vectors = model_directory + local_content_name + '.combined_vectors.txt'
combined_vectors_tsne = model_directory + local_content_name + '.combined_vectors_tsne.txt'

Global knowledge vectors -- wikipedia & Leipzig Corpora
-------------------------------------------------------
First step is to compute word embeddings of a global knowledge base (e.g. wikipedia or the Leipzig Corpora) to capture the generic meaning of words in widely used contexts.

The gensim package has examples of processing wikipedia dumps as well as streaming corpus implementation. The article just glosses over these steps and the sample github code grabs an undocumented data set from the authors drobbox account. In the cells below we rely on word2vec:
<pre>
git clone https://github.com/tmikolov/word2vec.git
</pre>
Also, in order to compute the t-sne embeddings with a c-language program, used bhtsne:
<pre>
git clone https://github.com/lvdmaaten/bhtsne.git
</pre>
When using jupyter-gallery docker image, usually install these in the /root directory. Hardwired into this notebook. 

**TODO:** 
* Parse the wikipedia dump name and use it as the prefix for the other intermediate files.
* Download a wikipedia dump if it doesn't already exist.
* Make things work for other languages (hundreds of wikipedias).
* Check that WikiCorpus does lowercase each word.
* Handle stopwords and substitution lists consistently.
* Stem global and local data sets.
* Check to see any value of using textblob over nltk.

### Process wikipedia dump
First download the wikipedia dump and place it in the data directory before running this notebook. The cell below will use the gensim class WikiCorpus to strip the wikipedia markup and store each article as one line of the output text file. Only do these computations once if possible.

In [ ]:
if not os.path.isfile(knowledge_base_text):
    space = ' '
    i = 0
    output = open(knowledge_base_text, 'w')
    logger.info('Processing knowledge base %s', knowledge_base)
    wiki = WikiCorpus(data_directory + knowledge_base, lemmatize=False, dictionary={})
    for text in wiki.get_texts():
        output.write(space.join(text) + "\n")
        i = i + 1
        if (i % 10000 == 0):
            logger.info("Saved " + str(i) + " articles")
    output.close()
    logger.info("Finished Saved " + str(i) + " articles")
else:
    logger.info('Knowledge base %s already on disk.', knowledge_base_text)

### Compute word vectors for knowledge base

In [ ]:
if not os.path.isfile(knowledge_base_vectors):
    knowledge_base_skipgram = fasttext.skipgram(knowledge_base_text, knowledge_base_vectors, lr=0.02, 
        dim=knowledge_base_vector_dimension, ws=5,
        epoch=1, min_count=5, neg=5, loss='ns', bucket=2000000, minn=3, maxn=6,
        thread=8, t=1e-4, lr_update_rate=100)
else:
    logger.info('Knowledge vectors %s already on disk.', knowledge_base_vectors)
    knowledge_base_skipgram = fasttext.load_model(knowledge_base_model)

Simple test to see if the model created/read ok.

In [ ]:
print u'siêu_thị' in knowledge_base_skipgram
print u'supermarket' in knowledge_base_skipgram

Create a counter to keep track of the knowledge base vocabulary. Later the sample code uses this to find the vocabulary in common between the knowledge base and the user generated data. Try to process both data sets in the same way.

In [ ]:
knowledge_base_exist = Counter()
for w in knowledge_base_skipgram.words:
    knowledge_base_exist[w.lower()] = w.lower()
knowledge_base_vocab_lowercase = knowledge_base_exist.keys()

In [ ]:
logger.info(u'siêu_thị: %s', knowledge_base_exist[u'siêu_thị'])
logger.info('funky: %s', knowledge_base_exist[u'funky'])
logger.info('san_diego: %s', knowledge_base_exist[u'san_diego'])

User content vectors -- OpenSubtitles2016
-----------------------------------------
OpenSubtitles is a very useful project for language analysis since it has a decent collection of parrallel sentences -- the foreign language captions that enthusiasts have created for their favorite movies.

---
Start with an `input.xml`, file listing captions from foreign film' obtained from the OpenSubtitle project. The final segmented text is input_text.txt.wseg (local_content_txt_sent_tkn_wseg). The parts of speech tagging is required in order to strip out the nouns (better labels for topics).

<pre>
BeautifulSoup:                     input.xml -> input.txt 
                           local_content_xml -> local_content_txt   
JVnSenSegmenter:                   input.txt -> input.txt.sent
                           local_content_txt -> local_content_txt_sent   
JVnTokenizer:                 input.txt.sent -> input.txt.sent.tkn
                      local_content_txt_sent -> local_content_txt_sent_tkn         
JVnSegmenter:             input.txt.sent.tkn -> input.txt.sent.tkn.wseg
                  local_content_txt_sent_tkn -> local_content_txt_sent_tkn_wseg  
POSTagging:          input.txt.sent.tkn.wseg -> input.txt.sent.tkn.wseg.pos
             local_content_txt_sent_tkn_wseg -> local_content_txt_sent_tkn_wseg_pos
</pre>

**Extract text from data**

Uses BeautifulSoup library to find all tags `'r'` and strip the text from them.

TODO:
1. Stream text through memory so that larger files can be proccessed.
2. Allow for a directory of subfiles.

In [ ]:
with open(local_content_xml,'r') as fp:
    soup = BeautifulSoup(fp,'lxml')

In [ ]:
with open(local_content_txt,'w') as fp:
    for s in soup.findAll('s'): 
        fp.write(s.text.strip().encode('utf-8') + '\n')

**Sentence Segmentation**

In [ ]:
cp = '/root/JVnTextPro/target/jvn-text-pro-2.0.jar:/root/.m2/repository/args4j/args4j/2.33/args4j-2.33.jar'
model_dir = '/root/JVnTextPro/models/jvnsensegmenter/'
java_class = 'jvnsensegmenter.JVnSenSegmenter'
cmd = 'java -cp {} {} -modeldir {} -inputfile {}'.format(cp, java_class, model_dir, local_content_txt)
output = os.popen(cmd)

**Sentence Tokenization**

Note: JVnTokenizer basically separates punctuation from words. Does not bother, for example, numbers like 22,216.

In [ ]:
cp = '/root/JVnTextPro/target/jvn-text-pro-2.0.jar:/root/.m2/repository/args4j/args4j/2.33/args4j-2.33.jar'
java_class = 'jvntokenizer.JVnTokenizer'
cmd = 'java -cp {} {} -inputfile {}'.format(cp, java_class, local_content_txt_sent)
output = os.popen(cmd)

**Word Segmentation**

In [ ]:
cp = '/root/JVnTextPro/target/jvn-text-pro-2.0.jar:/root/.m2/repository/args4j/args4j/2.33/args4j-2.33.jar'
model_dir = '/root/JVnTextPro/models/jvnsegmenter/'
java_class = 'jvnsegmenter.WordSegmenting'
cmd = 'java -cp {} {} -modeldir {}  -inputfile {}'.format(cp, java_class, model_dir, local_content_txt_sent_tkn)
output = os.popen(cmd)

Note: Seems to be a time delay before local_content_txt_sent_tkn_wseg is written to disk. Wait for it.

**Part of Speech Tagging**

In [ ]:
cp = '/root/JVnTextPro/target/jvn-text-pro-2.0.jar:/root/.m2/repository/args4j/args4j/2.33/args4j-2.33.jar'
model_dir = '/root/JVnTextPro/models/jvnpostag/maxent/'
java_class = 'jvnpostag.POSTagging'
cmd = 'java -cp {} {} -tagger maxent -modeldir {}  -inputfile {}'.format(cp, java_class, model_dir, 
                                                                         local_content_txt_sent_tkn_wseg)
output = os.popen(cmd)

**List of stop words**

The list below came from `elasticsearch` Vietnamese plugin.

In [ ]:
stopwords = ["bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện",
             "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến nỗi", "đều", "điều",
             "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "một_cách",
             "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", "qua", "ra",
             "rằng", "rằng", "rất", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", "trước",
             "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"]

**Read Segmented Senteces**

In [ ]:
## After processing, each line is a sentence. 
## Read in lines, skipping empty lines, to yield
text_lines = []
with open(local_content_txt_sent_tkn_wseg,'rb') as local_content_file:
    for line in local_content_file:
        line = line.strip()
        if line != '':
            text_lines.append(line)
for line in text_lines[0:3]:
    logger.info('%s', line)
logger.info("Text lines: %d", len(text_lines))

## The 
sentences = [[w for w in line.split()] for line in text_lines]
for sent in sentences[0:3]:
    logger.info('%s', sent)

**Vietnamese Parts of Speech**
1. N: Noun (danh từ)
2. Np: Personal Noun (danh từ riêng)
3. Nc: Classification Noun (danh từ chỉ loại)
4. Nu: Unit Noun (danh từ đơn vị)
5. V: verb (động từ)
6. A: Adjective (tính từ)
7. P: Pronoun (đại từ)
8. L: attribute (định từ)
9. M: Numeral (số từ)
10. R: Adjunct (phụ từ) 
11. E: Preposition (giới từ)
12. C: conjunction (liên từ)
13. I: Interjection (thán từ)
14. T: Particle, modal particle (trợ từ, tiểu từ)
15. B: Words from foreign countries (Từ mượn tiếng nước ngoài ví dụ Internet, ...)
16. Y: abbreviation (từ viết tắt)
17. X: un-known (các từ không phân loại được)
18. Mrk: punctuations (các dấu câu)

In [ ]:
def read_nouns_from_pos_data(path):
    with open(path, 'rU') as data:
        reader = csv.reader(data, delimiter=' ')
        for row in reader:
            nouns = []
            for field in row:
                try:
                    word,pos = field.split('/')
                    if pos in ['N','Np']: #,'Nc','Nu']:
                        nouns.append(word)
                except Exception as ex:
                    logger.error('%s %s', ex, field)
            yield nouns

In [ ]:
sentences_nouns = []
count = 0
for nouns in read_nouns_from_pos_data(local_content_txt_sent_tkn_wseg_pos): 
    if False and count > 10: 
        break
    count += 1
    sentences_nouns.append(nouns)
    
logger.info('%d', len(sentences_nouns))

Compute word vectors
--------------------

In [ ]:
if not os.path.isfile(local_content_vectors):
    local_content_skipgram = fasttext.skipgram(local_content_txt_sent_tkn_wseg, model_directory + local_content_name, 
        lr=0.02, dim=local_content_vector_dimension, ws=5,
        epoch=1, min_count=0, neg=5, loss='ns', bucket=2000000, minn=3, maxn=6,
        thread=8, t=1e-5, lr_update_rate=100)
else:
    logger.info('Local vectors %s already on disk.', local_content_vectors)
    local_content_skipgram = fasttext.load_model(local_content_model)

In [ ]:
logger.info('Creating word vecs')

words=[w for text in sentences_nouns for w in text]
Vocab=set(words)

model_comb={}
model_comb_vocab=[]

common_vocab=set(knowledge_base_vocab_lowercase).intersection(local_content_skipgram.words).intersection(Vocab)

for w in common_vocab:
    if len(w)>2:
        model_comb[w]=np.array(np.concatenate((knowledge_base_skipgram[w],local_content_skipgram[w])))
        model_comb_vocab.append(w)
        
logger.info('Length of common_vocab = %d', len(common_vocab))

In [ ]:
print len(set(knowledge_base_skipgram.words))
print len(set(local_content_skipgram.words))

In [ ]:
writer = csv.writer(open(combined_vectors,'w'),delimiter='\t')
for k in model_comb.keys():
    writer.writerow(model_comb[k])

In [ ]:
combined_vectors

In [ ]:
!/root/bhtsne/bhtsne.py -i {combined_vectors} -o {combined_vectors_tsne} -d 2 -v

In [ ]:
import pandas as pd
from bokeh.charts import Scatter, output_notebook, show

In [ ]:
output_notebook()

In [ ]:
reader = csv.reader(open(combined_vectors_tsne), delimiter='\t')
count = 0
X_full = []
for row in reader:
    if False and count >= 10000:
        break
    X_full.append(np.array([float(p) for p in row]))
    count += 1

In [ ]:
df = pd.DataFrame(X_full, columns=['x','y'])

In [ ]:
df.tail()

In [ ]:
p = Scatter(df, x='x', y='y', color='blue')
show(p)

In [ ]:
##Create a frequency count of words in email
words=[w for text in sentences_nouns for w in text]
Vocab=set(words)

In [ ]:
###Helper Functions
def norm(a):
    return np.sqrt(np.sum(np.square(a)))

def cosine(a,b):
    return 1-np.dot(a,b)/np.sqrt(np.sum(a**2)*np.sum(b**2))

def l1(a,b):
    return abs(a-b).sum()

def l2(a,b):
    return np.sqrt(np.square(a-b).sum())

In [ ]:
### Create a list of words to be clustered based on a model with some l2_threshold and can normalize the vectors 
### and also repeat or no
def create_word_list(model,vocab,features,Texts,repeat=True,l2_threshold=0,normalized=True,min_count=100,min_length=0):
    data_d2v=[]
    word_d2v=[]
    words_text=[w for text in Texts for w in text]
    count=Counter(words_text)
    if repeat:
        for text in Texts:
            for w in text:
                if w in vocab and count[w]>min_count:
                    if len(w)>min_length and l2(model[w],np.zeros(features))>l2_threshold:
                        if normalized:
                            data_d2v.append(model[w]/l2(model[w],np.zeros(features)))
                        else:
                            data_d2v.append(model[w])
                        word_d2v.append(w)
    else:
        A=set(words_text)
        for w in vocab:
            if w in A and len(w)>min_length and l2(model[w],np.zeros(features))>l2_threshold and count[w]>min_count:
                if normalized:
                    data_d2v.append(model[w]/l2(model[w],np.zeros(features)))
                else:
                    data_d2v.append(model[w])
                word_d2v.append(w)

    return data_d2v, word_d2v

In [ ]:
#Run Agglomerative clustering
logger.info('Clustering for depth...')
local_vec = True

data_d2v,word_d2v=create_word_list(model_comb,model_comb_vocab,25*local_vec+300,sentences_nouns,repeat=False,normalized=True,min_count=0,l2_threshold=0)
spcluster=fastcluster.linkage(data_d2v,method='average',metric='cosine')

In [ ]:
def calculate_depth(spcluster,words, num_points):
    cluster=[[] for w in xrange(2*num_points)]
    c=Counter()
    for i in xrange(num_points):
        cluster[i]=[i]

    for i in xrange(len(spcluster)):
        x=int(spcluster[i,0])
        y=int(spcluster[i,1])
        xval=[w for w in cluster[x]]
        yval=[w for w in cluster[y]]
        cluster[num_points+i]=xval+yval
        for w in cluster[num_points+i]:
            c[words[w]]+=1
        cluster[x][:]=[]
        cluster[y][:]=[]    
    return c

In [ ]:
##Calculate depth of words
num_points=len(data_d2v)
depth=calculate_depth(spcluster,word_d2v,num_points)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram

In [ ]:
plt.figure()
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    spcluster,
    truncate_mode='lastp',
    p=20,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=12.,  # font size for the x axis labels
    show_contracted=True,
)
plt.tight_layout()

In [ ]:
logger.info('Computing co-occurence graph')

T=[' '.join(w) for w in sentences_nouns]

In [ ]:
for line in T[0:10]: print line

In [ ]:
logger.info(len(T))

In [ ]:
##Co-occurence matrix
cv=CountVectorizer(token_pattern=u'(?u)\\b([^\\s]+)')
bow_matrix = cv.fit_transform(T)
id2word={}
for key, value in cv.vocabulary_.items():
    id2word[value]=key

ids=[]
for key,value in cv.vocabulary_.iteritems():
    if key in model_comb_vocab:
        ids.append(value)

sort_ids=sorted(ids)
bow_reduced=bow_matrix[:,sort_ids]
normalized = TfidfTransformer().fit_transform(bow_reduced)
similarity_graph_reduced=bow_reduced.T * bow_reduced

In [ ]:
##Depth-rank weighting of edges, weight of edge i,j=cosine of angle between them
logger.info('Computing degree')
m,n=similarity_graph_reduced.shape

cx=similarity_graph_reduced.tocoo()
keyz=[id2word[sort_ids[w]] for w in xrange(len(sort_ids))]
data=[]
ro=[]
co=[]
for i,j,v in itertools.izip(cx.row, cx.col, cx.data):
    if v>0 and i!=j:
        value=1
        if value>0:
            ro.append(i)
            co.append(j)
            data.append(value)

SS=sp.sparse.coo_matrix((data, (ro, co)), shape=(m,n))
SP_full=SS.tocsc()
id_word={w:id2word[sort_ids[w]] for w in xrange(len(sort_ids))}
word_id={value:key for key,value in id_word.items()}

In [ ]:
logger.info('Computing metrics')
#compute metrics
degsum=SP_full.sum(axis=1)
deg={}
for x in xrange(len(sort_ids)):
    deg[id2word[sort_ids[x]]]=int(degsum[x])

max_deg=max(deg.values())
max_depth=max(depth.values())

temp_deg_mod={w:np.log(1+deg[w])/np.log(1+max_deg) for w in deg.iterkeys()}
alpha=np.log(0.5)/np.log(np.median(temp_deg_mod.values()))
deg_mod={key:value**alpha for key, value in temp_deg_mod.iteritems()}

temp={key:value*1./max_depth for key, value in depth.iteritems()}
alpha=np.log(0.5)/np.log(np.median(temp.values()))
depth_mod={key:value**alpha for key, value in temp.iteritems()}

temp={key:deg_mod[key]*depth_mod[key] for key in depth_mod.iterkeys()}
max_metric=np.max(temp.values())
metric={key:value*1./max_metric for key,value in temp.iteritems()}

In [ ]:
logger.info('max_deg = %s, max_depth = %s',max_deg, max_depth)

In [ ]:
##Kmeans
NUM_TOPICS = 150
K=NUM_TOPICS
kmeans=KMeans(n_clusters=K)
kmeans.fit([w for w in data_d2v])
kmeans_label={word_d2v[x]:kmeans.labels_[x] for x in xrange(len(word_d2v))}

kmeans_label_ranked={}

topic=[[] for i in xrange(K)]
clust_depth=[[] for i in xrange(K)]
for i in xrange(K):
    topic[i]=[word_d2v[x] for x in xrange(len(word_d2v)) if kmeans.labels_[x]==i]
    temp_score=[metric[w] for w in topic[i]]
    clust_depth[i]=-np.mean(sorted(temp_score,reverse=True)[:])#int(np.sqrt(len(topic[i])))])
index=np.argsort(clust_depth)
for num,i in enumerate(xrange(K)):
    for w in topic[index[i]]:
        kmeans_label_ranked[w]=i

In [ ]:
logger.info('Done...Generating output')
lister=[]
to_show=K
to_show_words=20 #the maximum number of words of each type to display
for i in xrange(to_show):
    top=topic[index[i]]
    sort_top=[w[0] for w in sorted([[w,metric[w]] for w in top],key=itemgetter(1),reverse=True)]
    lister.append(['Topic %d' %(i+1)]+sort_top[:to_show_words])

max_len=max([len(w) for w in lister])
new_list=[]
for list_el in lister:
    new_list.append(list_el + [''] * (max_len - len(list_el)))
Topics=list(itertools.izip_longest(*new_list))
#X.insert(len(X),[-int(clust_depth[index[w]]*100)*1./100 for w in xrange(K)])
sorted_words=[w[0] for w in sorted(metric.items(),key=itemgetter(1),reverse=True)][:to_show_words]

In [ ]:
score_words = sorted_words
deep_words = [w[0] for w in depth.most_common(to_show_words)]
filer = 'wiki_simple.txt'
outfile_topics = data_directory + filer.split('.')[0] + '_topics.csv'
outfile_score = data_directory + filer.split('.')[0] + '_score.csv'
outfile_depth = data_directory + filer.split('.')[0] + '_depth.csv'
b = open(outfile_topics, 'wb')
a = csv.writer(b)
a.writerows(Topics)
b = open(outfile_score, 'wb')
a = csv.writer(b)
a.writerows([[w] for w in score_words])
b = open(outfile_depth, 'wb')
a = csv.writer(b)
a.writerows([[w] for w in deep_words])

In [ ]:
score_words[0:10]

In [ ]:
step = 10
for j in range(K/step + 1):
    first = j*step + 1;last = j*step + step
    print 'Total number of Topics = {}. Displaying Topics {} thru {}.'.format(K, first, last)
    print tabulate([Topics[i][first-1:last] for i in range(0,21)], tablefmt=u'simple')